# Evaluator Codegen Tutorial

Code generation is the core of SymForce - you provide a Values that describes the symbolic inputs to your computation, a Values that gives the desired output expressions, and it generates an executable package to efficiently evaluate that expression. So far Python and C++ packages are supported, but it is simple to add more.

The executable packages are standalone and do not depend on SymForce. They contain type structs generated from the input/output Values, implementations of the concepts, and an Evaluator class to execute the generated code.

Let's set up an example for the [double pendulum](https://www.myphysicslab.com/pendulum/double-pendulum-en.html). We'll skip the derivation and just define the equations of motion for the angular acceleration of the two links:

In [ ]:
# Setup
import os
import symforce
symforce.set_backend('sympy')
symforce.set_log_level('warning')

from symforce import geo
from symforce import sympy as sm
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [ ]:
# Define symbols
L = geo.V2().symbolic('L').T      # Length of the two links
m = geo.V2().symbolic('m').T      # Mass of the two links
ang = geo.V2().symbolic('a').T    # Angle of the two links
dang = geo.V2().symbolic('da').T  # Angular velocity of the two links
g = sm.Symbol('g')                # Gravity

In [ ]:
# Angular acceleration of the first link
ddang_0 = (
    -g * (2 * m[0] + m[1]) * sm.sin(ang[0])
    - m[1] * g * sm.sin(ang[0] - 2 * ang[1])
    - 2
    * sm.sin(ang[0] - ang[1])
    * m[1]
    * (dang[1] * 2 * L[1] + dang[0] * 2 * L[0] * sm.cos(ang[0] - ang[1])))\
    / (L[0] * (2 * m[0] + m[1] - m[1] * sm.cos(2 * ang[0] - 2 * ang[1])))
display(ddang_0)

In [ ]:
# Angular acceleration of the second link
ddang_1 = (
    2
    * sm.sin(ang[0] - ang[1])
    * (
        dang[0] ** 2 * L[0] * (m[0] + m[1])
        + g * (m[0] + m[1]) * sm.cos(ang[0])
        + dang[1] ** 2 * L[1] * m[1] * sm.cos(ang[0] - ang[1])
    )
) / (L[1] * (2 * m[0] + m[1] - m[1] * sm.cos(2 * ang[0] - 2 * ang[1])))
display(ddang_1)

Now let's organize the input symbols into a Values hierarchy:

In [ ]:
inputs = Values()

inputs['ang'] = ang
inputs['dang'] = dang
    
with inputs.scope('constants'):
    inputs['g'] = g

with inputs.scope('params'):
    inputs['L'] = L
    inputs['m'] = m

inputs

The output will simply be a 2-vector of the angular accelerations:

In [ ]:
outputs = Values(
    ddang=geo.V2(ddang_0, ddang_1)
)

Now run code generation to produce an executable module (in a temp directory if none provided):

In [ ]:
from symforce.codegen import EvaluatorCodegen

codegen = EvaluatorCodegen(inputs, outputs, 'double_pendulum')
py_data = codegen.generate()

# Print what we generated
import os
package_dir = py_data['package_dir']
print('Files generated in package {}:\n'.format(package_dir))
for f in py_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, package_dir)))

That's it. You can see in the list above that we produced input types for each Values object (including nested ones), a StorageOps implementation, an Evaluator class to execute the code, and an example usage script.

Let's look at the `input_t` type that was generated from the input `Values`. It's a very basic struct type that contains the values keys as attributes. Note that this backend uses numpy types.

In [ ]:
display_code_file(os.path.join(package_dir, 'types', 'input_t.py'), 'python')

The StorageOps implementation for this type goes back and forth to a Python list:

In [ ]:
display_code_file(os.path.join(package_dir, 'storage_ops', 'input_t.py'), 'python')

Now the evaluator code, which computes the output expression and uses the StorageOps methods as helpers:

In [ ]:
display_code_file(os.path.join(package_dir, 'evaluator.py'), 'python')

The `_execute()` method above computes the output expression. It operates on lists rather than the types, which makes code generation across languages more uniform. Note how there are `tmp` variables for shared intermediate values. This is a result of [common sub-expression elimination](https://en.wikipedia.org/wiki/Common_subexpression_elimination), which is a key step to efficient execution. As problems become large, sharing sub-expressions between all output values is a huge advantage of symbolic computation combined with code generation.

SymPy itself has comprehensive code generation [tools](https://docs.sympy.org/latest/modules/codegen.html), which SymForce can leverage under the hood. However, it is easy to hit bottlenecks and hard to scale up both in terms of organization and performance. SymForce is optimized for these two things:

1) Generate production-ready code out of the box  
2) Scale up to large problems efficiently  

## Execution

Execution is a matter of instantiating the generated `Evaluator` class and calling it with an `input_t` type. The generated example is directly runnable and demonstrates this. The only dependency is numpy for Python and the standard library for C++ generated code. They do **not** depend on symforce.

Normally we would import the package generated on disk by adding the directory to the path. For Python, SymForce dynamically loads the package for you and provides the evaluator class:

In [ ]:
evaluator = py_data['evaluator']
evaluator

Let's create an input type and fill its values:

In [ ]:
import numpy as np

inp = evaluator.input_t()
inp.constants.g = 9.81               # [m/s^2]
inp.params.L = np.array([0.5, 0.8])  # [m]
inp.params.m = np.array([0.5, 1.1])  # [kg]
inp.ang = np.array([0.0, 0.3])       # [rad]
inp.dang = np.array([0.0, 0.0])      # [rad/s]
inp

Execute to get the outputs:

In [ ]:
# Execute
out = evaluator.execute(inp)
out

For fun, simulate five seconds of the pendulum and plot the angles:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def simulate(inp, out, dt):
    inp.dang += out.ddang * dt
    inp.ang += inp.dang * dt

steps = 1000
ts = np.linspace(0, 5.0, num=steps)
angles = np.zeros((steps, 2))
for i in range(len(ts) - 1):
    out = evaluator.execute(inp)
    simulate(inp, out, dt=ts[i + 1] - ts[i])
    angles[i, :] = inp.ang

plt.plot(ts, angles);

## C++ Code Generation

In [ ]:
Code generation works the same in every language, we just specify the mode argument:

In [ ]:
from symforce.codegen import EvaluatorCodegen
from symforce.codegen import CodegenMode

codegen = EvaluatorCodegen(inputs, outputs, 'double_pendulum')
cpp_data = codegen.generate(mode=CodegenMode.CPP)

# Print what we generated
import os
package_dir = cpp_data['package_dir']
print('Files generated in package {}:\n'.format(package_dir))
for f in cpp_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, package_dir)))

Let's look at a type:

In [ ]:
display_code_file(os.path.join(package_dir, 'types', 'input_t.h'), 'C++')

StorageOps are done through template specialization:

In [ ]:
display_code_file(os.path.join(package_dir, 'storage_ops', 'input_t.h'), 'C++')

The generated code is separated into a .cc file of the Evaluator:

In [ ]:
display_code_file(os.path.join(package_dir, 'evaluator.cc'), 'C++')

Let's look at the usage example:

In [ ]:
display_code_file(os.path.join(package_dir, 'example', 'example.cc'), 'C++')

It also comes with a basic Makefile to build it, assuming `clang` is available by default:

In [ ]:
display_code_file(os.path.join(package_dir, 'example', 'Makefile'), 'make')

We can use subprocess to compile and run the example. Note that it prints a nan because of divison by zero, since the inputs are mostly filled with zeros. There is a technique not covered here for ever avoiding nans.

In [ ]:
import subprocess
out = subprocess.check_output(['make', '-C', os.path.join(package_dir, 'example')])
print(out.decode())

In [ ]:
out = subprocess.check_output([os.path.join(package_dir, 'example', 'example')])
print(out.decode())

## Advanced Topics

TODO: Handling singularities and NaNs  
TODO: Conditional logic  
TODO: Optimization  
TODO: Manifolds and Lie groups  
TODO: Geometric algebra  

## Cleanup

In [ ]:
from symforce import python_util
python_util.remove_if_exists(py_data['package_dir'])
python_util.remove_if_exists(cpp_data['package_dir'])